In [ ]:
#function check เรียนไปแล้วหรือยัง (ตาราง subject)
from app_users.models import Subjects

def check_pass_subject(request, user_id, sub_id):
    data = Subjects.objects.filter(userid = user_id) #เอาทุกอย่างในตารางsubjectsของ Userคนนั้น
    p_sub = data.values_list('real_subject_id', flat=True) #เอาเฉพาะวิชา
    for i in p_sub:
        if sub_id in i :
            return False
    else :
        return True

In [11]:
#function check เรียนไปแล้วหรือยัง (ตาราง subject)
import sqlite3

def check_pass_subject(sub_id,u_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    #เลือกวิชาทั้งหมดที่มีจาก subjects ของ user คนนั้น
    c.execute("SELECT real_subject_id FROM subjects WHERE UserID = ?", (u_id,))
    total_subjects = c.fetchall()
    print(total_subjects)
    
    #ไปค้นหารหัสวิชาจาก sub_id ที่ user ป้อนเข้ามา
    c.execute("SELECT code FROM app_schedule_subjects_info WHERE ID = ?", (sub_id,))
    select_sub = c.fetchall()
    print(select_sub[0])

    # Close the connections
    conn.close()

    if select_sub[0] in total_subjects:
        return False
    else:
        return True

check_pass_subject(600,1)

[('010113030',), ('010123102',), ('040203111',), ('040313005',), ('040313006',), ('080103001',), ('080303502',), ('080303601',), ('010113138',), ('010113139',), ('010123103',), ('040003004',), ('040203112',), ('040313007',), ('040313008',), ('080103002',), ('010123105',), ('010123106',), ('010123107',), ('010123108',), ('010123131',), ('010123133',), ('080103034',)]
('040503001',)


True

In [7]:
from app_users.models import Subjects

def check_credit(request, user_id, sub_id):
    data = Subjects.objects.filter(userid = user_id) #เอาทุกอย่างในตารางsubjectsของ Userคนนั้น
    p_sub = data.values_list('credit', flat=True) #เอาเฉพาะหน่วยกิต
    total_credits = 0
    for i in p_sub:
        total_credits+=i
    
    data2 = Subjects.objects.filter(real_subject_id = sub_id)#เอาSubjectที่userที่เลือก
    p_sub2 = data.values_list('credit', flat=True)#เอาหน่วยกิต
    subject_credits = p_sub2[0]
    credits_now = total_credits + subject_credits
    print(credits_now)
    if credits_now > 22 :
        return False
    else :
        return True

ModuleNotFoundError: No module named 'django'

In [ ]:
#Function check credit
import sqlite3
def check_credit(user_id,sub_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    # Execute the SELECT statement and retrieve the total sum of credits
    c.execute("SELECT SUM(app_schedule_subjects_info.credit) FROM app_schedule_subjects_info,app_schedule_user_subjects "+
              "WHERE app_schedule_user_subjects.user_id_id = ?  AND app_schedule_user_subjects.sub_id_id = app_schedule_subjects_info.ID ",(user_id,))
    result = c.fetchone()

    # Get the total sum of credits from the result tuple
    total_credits = result[0]
 
    # Print the total sum of credits
    print("Total credits:", total_credits)

    # Execute the SELECT statement to retrieve the sum of credits for the specified subject
    c.execute("SELECT credit FROM app_select_subjects_test_date WHERE code = ?", (sub_id,))
    result = c.fetchone()
    print(result)
    # Get the sum of credits for the specified subject from the result tuple
    subject_credits = result[0]

    # Print the sum of credits for the specified subject
    print(f"Subject {sub_id} credits:", subject_credits)

    # Close the connections
    conn.close()
    
    credits_now = total_credits + subject_credits

    if credits_now >22:
        return False
    else:
        return True

check_credit(1,"010113138")


Total credits: 13
(3,)
Subject 010113138 credits: 3


True

In [10]:
#function check ลงไปแล้วรึยัง (ตารางเก็บวิชาที่ผู้ใช้เลือก)
import sqlite3
def check_regis_subject(sub_id,u_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    #เลือกวิชาทั้งหมดที่มีจาก select_subjects ของ user คนนั้น
    c.execute("SELECT sub_id_id FROM app_schedule_user_subjects WHERE user_id_id = ?", (u_id,))
    data = c.fetchall()
    print(data)

    # Close the connections
    conn.close()

    if (sub_id,) in data :
        return False
    return True

check_regis_subject(500,1)

[]


True

In [ ]:
# add 0 front of subject_code
import sqlite3
user_id = 8
conn = sqlite3.connect("w3.db")
c = conn.cursor()
c.execute("SELECT * FROM subjects WHERE UserID = ?",(user_id,))
result = c.fetchall()
for a in result:
    x = '0'+a[1]
    print(x)
    c.execute("UPDATE subjects SET real_subject_id = ? WHERE UserID = ? AND real_subject_id = ? ",(x,user_id,a[1],))

conn.commit()
conn.close()

010113030
010123102
040203111
040313005
040313006
080103001
080303502
080303601
010113138
010113139
010123103
040003004
040203112
040313007
040313008
080103002
010123105
010123106
010123107
010123108
010123131
010123133
080103034


In [ ]:
#function check วันเวลาเรียน
import sqlite3
from datetime import datetime

def check_time_study(sub_id,user_id):
      conn = sqlite3.connect("w3.db")
      c = conn.cursor()
      c.execute("SELECT sub_id_id FROM app_schedule_user_subjects WHERE user_id_id = ? ",(user_id,)) 
      allsub = c.fetchall()
      print(allsub)

      alltime = []
      for x in allsub:
            sub = x[0]
            c.execute("SELECT day,start_time,end_time FROM app_schedule_subjects_info WHERE ID = ? ",(sub,))
            time = c.fetchall()
            alltime.append(time[0])

      c.execute("SELECT day,start_time,end_time FROM app_schedule_subjects_info WHERE ID = ? ",(sub_id,))
      subnow = c.fetchall()
      print(subnow)

      for y in alltime:
            print(y[0],y[1],y[2])
            daynow = subnow[0][0]
            startnow = subnow[0][1].split(':')[0]
            endnow = subnow[0][2].split(':')[0]

            dayold = y[0]
            startold = y[1].split(':')[0]
            endold = y[2].split(':')[0]
            if daynow == dayold:
                  if endnow <= startold or endold <= startnow: 
                        return True
                  else:
                        return False
      return True
            
   


check_time_study("444",1)

[(430,), (431,), (443,), (446,)]
[('W', '13:00:00', '16:00:00')]
M 13:00:00 16:00:00
H 09:00:00 12:00:00
F 10:00:00 12:00:00
H 13:00:00 16:00:00


True

In [ ]:
#function check วันเวลาสอบกลางภาค
from datetime import datetime
def check_time_midterm(sub_id,user_id):
    conn = sqlite3.connect("w3.db")
    c = conn.cursor()

    c.execute("SELECT user.sub_id_id, testdate.mid_numday ,testdate.mid_starttime,testdate.mid_endtime "+
              "FROM app_schedule_user_subjects AS user ,app_select_subjects_test_date AS testdate WHERE user.user_id_id = ?"+
               "AND user.sub_id_id = testdate.code ",(user_id,)) 
    allsub = c.fetchall()
    for x  in allsub:
        sub = x[0]
        c.execute

In [ ]:
#function check วันเวลาปลายภาค
from datetime import datetime
def check_time_final(sub_id,user_id):

In [ ]:
from datetime import datetime

event1_start = datetime(2023, 3, 10, 14, 0) # March 10th, 2023 at 2:00 PM
event1_end = datetime(2023, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM

event2_start = datetime(2023, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM
event2_end = datetime(2023, 3, 10, 17, 0) # March 10th, 2023 at 5:00 PM


In [ ]:
from datetime import datetime

event1_start = datetime(2566, 3, 10, 14, 0) # March 10th, 2023 at 2:00 PM
event1_end = datetime(2566, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM

event2_start = datetime(2566, 3, 10, 16, 0) # March 10th, 2023 at 4:00 PM
event2_end = datetime(2566, 3, 10, 17, 0) # March 10th, 2023 at 5:00 PM


In [ ]:
#function check วันเวลา
def check_time_overlap(event1_start, event1_end, event2_start, event2_end):
    if event1_end <= event2_start or event2_end <= event1_start:
        return False
    else:
        return True

if check_time_overlap(event1_start, event1_end, event2_start, event2_end):
    print("The two events overlap.")
else:
    print("The two events do not overlap.")


The two events do not overlap.
